https://www.dataquest.io/blog/settingwithcopywarning/  
の内容を検証するためのNotebook。


http://www.modelingonlineauctions.com/datasets  
の中にある「Xbox 3-day auctions.csv」をダウンロードしたい。  
しかし、使用しているOSがWindowsなので、!wget も !curlも使えない。  
よってpythonの機能でダウンロードしよう。  
……と思ったら、pandasのread_csvにURLを渡してやればできるんだな。簡単な話だった。

In [1]:
import pandas as pd

In [2]:
csv_file_path = 'xbox-3-day-auctions.csv'

if False: # todo: あとで条件を「csvファイルが存在しないときは」にしておこう
    url = 'http://www.modelingonlineauctions.com/datasets/Xbox%203-day%20auctions.csv?attredirects=0&d=1'
    data = pd.read_csv(url)
    data.to_csv(csv_file_path)
else:
    data = pd.read_csv(csv_file_path)

In [3]:
data.head()

,Unnamed: 0,auctionid,bid,bidtime,bidder,bidderrate,openbid,price
0,0,8213034705,95.0,2.927373,jake7870,0,95.0,117.5
1,1,8213034705,115.0,2.943484,davidbresler2,1,95.0,117.5
2,2,8213034705,100.0,2.951285,gladimacowgirl,58,95.0,117.5
3,3,8213034705,117.5,2.998947,daysrus,10,95.0,117.5
4,4,8213060420,2.0,0.065266,donnie4814,5,1.0,120.0


# Chained Assignment

In [4]:
data[data.bidder == 'parakeet2004']

,Unnamed: 0,auctionid,bid,bidtime,bidder,bidderrate,openbid,price
6,6,8213060420,3.00,0.186539,parakeet2004,5,1.0,120.0
7,7,8213060420,10.00,0.186690,parakeet2004,5,1.0,120.0
8,8,8213060420,24.99,0.187049,parakeet2004,5,1.0,120.0


In [5]:
data[data.bidder == 'parakeet2004']['bidderrate'] = 100

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


**SettingWithCopyWarning** が発生した。  
今回の場合、元の表の値は変わっていない。（意図したとおりに動いていない。）

In [6]:
data[data.bidder == 'parakeet2004']

,Unnamed: 0,auctionid,bid,bidtime,bidder,bidderrate,openbid,price
6,6,8213060420,3.00,0.186539,parakeet2004,5,1.0,120.0
7,7,8213060420,10.00,0.186690,parakeet2004,5,1.0,120.0
8,8,8213060420,24.99,0.187049,parakeet2004,5,1.0,120.0


**警告に対処するには、chained assignmentを回避してlocメソッドを使う。**

In [7]:
# .locを使って、新たな値を設定する
data.loc[data.bidder == 'parakeet2004', 'bidderrate'] = 100

# 結果を確認する。bidderrateの値が変更されている
data[data.bidder == 'parakeet2004']['bidderrate']

6    100
7    100
8    100
Name: bidderrate, dtype: int64

# Hidden chaining

In [8]:
winners = data.loc[data.bid == data.price]
winners.head()

,Unnamed: 0,auctionid,bid,bidtime,bidder,bidderrate,openbid,price
3,3,8213034705,117.5,2.998947,daysrus,10,95.00,117.5
25,25,8213060420,120.0,2.999722,djnoeproductions,17,1.00,120.0
44,44,8213067838,132.5,2.996632,*champaignbubbles*,202,29.99,132.5
45,45,8213067838,132.5,2.997789,*champaignbubbles*,202,29.99,132.5
66,66,8213073509,114.5,2.999236,rr6kids,4,1.00,114.5


In [9]:
#続けて何らかの処理を行っているとする
mean_win_time = winners.bidtime.mean()
... # 20 lines of code
mode_open_bid = winners.openbid.mode()

In [10]:
winners.loc[304]

Unnamed: 0           304
auctionid     8213922989
bid                   93
bidtime           2.9716
bidder               NaN
bidderrate             2
openbid             0.95
price                 93
Name: 304, dtype: object

In [11]:
winners.loc[304, 'bidder']

nan

In [12]:
# この入札者の真のユーザー名を知っているものと仮定して、データを変更してみよう

winners.loc[304, 'bidder'] = 'therealname'

c:\program files\python37\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


再び**SettingWithCopyWarning** が発生した。  
値を確認してみると、今回は意図したとおりに値が変更されている。

In [13]:
winners.loc[304, 'bidder']

'therealname'

**警告に対処するには、Dataframeのコピーを作ることを明示的に示せばよい。すなわち、copyメソッドを使う。**

In [14]:
winners = data.loc[data.bid == data.price].copy()
winners.loc[304, 'bidder'] = 'therealname'
print(winners.loc[304, 'bidder'])
print(data.loc[304, 'bidder']) # コピーなので元々のデータは変更されない

therealname
nan


# Tips and tricks for dealing with SettingWithCopyWarning

In [15]:
# 警告をオフにする

pd.set_option('mode.chained_assignment', None)
data[data.bidder == 'parakeet2004']['bidderrate'] = 100
# 連鎖インデックスをしても警告は全く発生しない

In [16]:
# 警告ではなくエラーが発生するように、設定を変更する

pd.set_option('mode.chained_assignment', 'raise')
data[data.bidder == 'parakeet2004']['bidderrate'] = 100
# エラーが発生する

SettingWithCopyError: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

In [17]:
# 直前の設定をリセット
pd.reset_option('mode.chained_assignment')

print(pd.get_option('mode.chained_assignment'))

with pd.option_context('mode.chained_assignment', None):
    data[data.bidder == 'parakeet2004']['bidderrate'] = 100

warn
